In [105]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim library
from modsim import *

# set the random number generator
np.random.seed(7)
import math
import pandas as pd

In [106]:
limeshare = State(number_of_scooters = 20, av_batt_life = 100, size_of_region = 2, send_truck = -1)
limeshare

,values
number_of_scooters,20
av_batt_life,100
size_of_region,2
send_truck,-1


In [107]:
def step(state, hour):
#     find number of users per hour
    demand = calcDemand(state, hour)
    
#     find the av time of use per rider from number of users found above
    length_of_use = calcLengthUsed(state, hour)
    
#     calc change in battery 
    calcBattDelta(state, demand, length_of_use)
    
#     find hour to send truck
    if(state.av_batt_life < 20 and state.send_truck == -1):
        sendTruck(state, hour)


In [108]:
def calcDemand(state, hour):
#     based on time of hour with higher prob in noon 4 traffic rates with sets time of day with max number of scooters
    return int(state.number_of_scooters / 10) * 3;

def calcLengthUsed(state, hour):
#     based on time used realtive to region, hour, demand with max av battry life 
    return state.size_of_region * 10
        
def calcBattDelta(state, demand, length_of_use):
    s = state.number_of_scooters 
    c = state.av_batt_life
    t = demand
    
    state.av_batt_life =  math.floor(((s-t)*c + t*(c- length_of_use)) / s)
        

def sendTruck(state, hour):
    state.send_truck = hour;

In [109]:
def run_simulation(state, num_steps):     
    results = TimeSeries()
    for i in range(num_steps):
        #  this is our line in the sand to prevent calculating after our desired purpose for the model has been found
        if(state.send_truck != -1):
            return
        #  adding one for first hour, second hour (most people don't talk hour zero) 
        # also we tho militray time is 00:00 -23:59 we would need exception handling in certain funcs
        step(state, i + 1)
        results[i] = state.av_batt_life

In [110]:
limeshare = State(number_of_scooters = 20, av_batt_life = 100, size_of_region = 3, send_truck = -1)
run_simulation(limeshare, 24)
limeshare

,values
number_of_scooters,20
av_batt_life,19
size_of_region,3
send_truck,9


In [111]:
def runModel():
#     tuple array ( size_of_region, send_truck) with fixed number_of_scooters at 20
    regions = linspace(1, 5, 10)
    results = []
    for i in regions:
        limeshare = State(number_of_scooters = 20, av_batt_life = 100, size_of_region = round(i + 1, 2), send_truck = -1)
        run_simulation(limeshare, 24)
        data = (limeshare.size_of_region, limeshare.send_truck)
        results.append(data)
    return results 

In [112]:
sample_regions = runModel()

[(2.0, 14.0),
 (2.44, 11.0),
 (2.89, 9.0),
 (3.33, 9.0),
 (3.78, 7.0),
 (4.22, 7.0),
 (4.67, 6.0),
 (5.11, 6.0),
 (5.56, 5.0),
 (6.0, 5.0)]

In [113]:
def runModel():
#     tuple array (number_of_scooters, send_truck) with fixed size_of_region at 2
    scooters = linspace(20, 70, 10)
    results = []
    for i in scooters:
        limeshare = State(number_of_scooters = int(i), av_batt_life = 100, size_of_region = 2, send_truck = -1)
        run_simulation(limeshare, 24)
        data = (limeshare.number_of_scooters, limeshare.send_truck)
        results.append(data)
    return results 

In [114]:
sample_scooters = runModel()

[(20, 14),
 (25, 17),
 (31, 14),
 (36, 17),
 (42, 14),
 (47, 14),
 (53, 14),
 (58, 14),
 (64, 14),
 (70, 14)]

In [115]:
region_data = linspace(1, 5, 10)


array([1.        , 1.44444444, 1.88888889, 2.33333333, 2.77777778,
       3.22222222, 3.66666667, 4.11111111, 4.55555556, 5.        ])

In [116]:
scooter_data = linspace(20, 70, 10)

array([20.        , 25.55555556, 31.11111111, 36.66666667, 42.22222222,
       47.77777778, 53.33333333, 58.88888889, 64.44444444, 70.        ])

In [117]:
truck_data = [[0 for x in region_data] for y in scooter_data] 

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [120]:
def runModel():
    for row, reigon in enumerate(region_data):
        for col, scooters in enumerate(scooter_data):
            limeshare = State(number_of_scooters = scooters, av_batt_life = 100, size_of_region = reigon, send_truck = -1)
            run_simulation(limeshare, 24)
            truck_data[row][col] = limeshare.send_truck

runModel()
truck_data

[[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0],
 [17.0, 21.0, 17.0, 21.0, 17.0, 21.0, 17.0, 21.0, 17.0, 17.0],
 [14.0, 17.0, 14.0, 17.0, 14.0, 17.0, 14.0, 17.0, 14.0, 14.0],
 [12.0, 14.0, 12.0, 14.0, 12.0, 14.0, 12.0, 14.0, 12.0, 12.0],
 [9.0, 12.0, 9.0, 12.0, 11.0, 12.0, 11.0, 11.0, 11.0, 9.0],
 [9.0, 11.0, 9.0, 11.0, 9.0, 9.0, 9.0, 9.0, 9.0, 9.0],
 [8.0, 9.0, 8.0, 9.0, 8.0, 9.0, 8.0, 9.0, 8.0, 8.0],
 [7.0, 9.0, 7.0, 8.0, 7.0, 8.0, 7.0, 8.0, 7.0, 7.0],
 [6.0, 8.0, 6.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 6.0],
 [6.0, 7.0, 6.0, 7.0, 6.0, 7.0, 6.0, 7.0, 6.0, 6.0]]

In [ ]:
# My reaction when the above code printed "HOLY SHIT!"

In [127]:
new_dataFrame = pd.DataFrame(data=truck_data, index=round(region_data, columns=scooter_data)
new_dataFrame

,20.000000,25.555556,31.111111,36.666667,42.222222,47.777778,53.333333,58.888889,64.444444,70.000000
1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1.444444,17.0,21.0,17.0,21.0,17.0,21.0,17.0,21.0,17.0,17.0
1.888889,14.0,17.0,14.0,17.0,14.0,17.0,14.0,17.0,14.0,14.0
2.333333,12.0,14.0,12.0,14.0,12.0,14.0,12.0,14.0,12.0,12.0
2.777778,9.0,12.0,9.0,12.0,11.0,12.0,11.0,11.0,11.0,9.0
3.222222,9.0,11.0,9.0,11.0,9.0,9.0,9.0,9.0,9.0,9.0
3.666667,8.0,9.0,8.0,9.0,8.0,9.0,8.0,9.0,8.0,8.0
4.111111,7.0,9.0,7.0,8.0,7.0,8.0,7.0,8.0,7.0,7.0
4.555556,6.0,8.0,6.0,7.0,7.0,7.0,7.0,7.0,7.0,6.0
5.000000,6.0,7.0,6.0,7.0,6.0,7.0,6.0,7.0,6.0,6.0
